In [20]:
from utils import *
import pandas as pd
import datetime as dt

opt_data = pd.read_csv('./data/options.csv')
opt_data["Expiration_date"] = pd.to_datetime(opt_data["Expiration_date"]).dt.tz_localize(None)
opt_data["lastTradeDate"] = pd.to_datetime(opt_data["lastTradeDate"]).dt.tz_localize(None)

opt_data = opt_data[["lastTradeDate", "Expiration_date", "strike", "lastPrice"]]
opt_data["days_to_expiration"] = (opt_data["Expiration_date"] - opt_data['lastTradeDate']).dt.days
opt_data

,lastTradeDate,Expiration_date,strike,lastPrice,days_to_expiration
0,2024-11-22 20:58:37,2024-12-20,30.0,26.70,27
1,2024-11-21 14:34:42,2024-12-20,36.0,19.40,28
2,2024-11-19 18:49:06,2024-12-20,37.0,17.00,30
3,2024-11-21 17:45:27,2024-12-20,38.0,18.20,28
4,2024-11-22 20:42:42,2024-12-20,40.0,16.82,27
...,...,...,...,...,...
247,2024-11-22 20:47:32,2027-01-15,80.0,22.40,783
248,2024-11-22 16:44:42,2027-01-15,85.0,20.60,783
249,2024-11-22 19:30:16,2027-01-15,90.0,20.60,783
250,2024-11-22 19:55:12,2027-01-15,95.0,20.45,783


In [ ]:
mi, sigma, r = pd.read_csv('./data/parameters.csv').values[0]

opt_data["price BS"] = opt_data.apply(lambda x: black_scholes(x["lastPrice"], x["strike"], x["days_to_expiration"], r, mi, sigma), axis=1)

(0.0015096225074639, 0.0344379853159919, 0.00041957)